In [264]:
% reset
% config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import scipy as sp
from statsmodels.api import add_constant
from statsmodels.regression import linear_model

% matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 6
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Tahoma']
sns.set(style='whitegrid', context ='paper')

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [340]:
# load the data
file = 'ei_models.csv'
df = pd.read_csv(file)

np.shape(df)

(946, 4)

In [266]:
# just making sure our results are the same

Y = df['slope']

# full model
X = df[['E', 'I', 'EI']]
X = add_constant(X)
ols_fit = linear_model.OLS(Y, X).fit()
e_i_ei_real = ols_fit.rsquared_adj

# e & ei model
X = df[['E', 'EI']]
X = add_constant(X)
ols_fit = linear_model.OLS(Y, X).fit()
e_ei_real = ols_fit.rsquared_adj


# i & ei model
X = df[['I', 'EI']]
X = add_constant(X)
ols_fit = linear_model.OLS(Y, X).fit()
i_ei_real = ols_fit.rsquared_adj

e_i_ei_real, e_ei_real, i_ei_real

(0.24121797529045896, 0.087520681201372685, 0.23791594155153151)

In [306]:
# resample all the i-model data to build up distributions

data_length = np.shape(df)[0]
subsample_size = 25 # grab <subsample_size> rows of data
surrogate_runs = 500 # get this many adjusted R^2

# initialize
e_i_ei_surr = np.zeros((surrogate_runs, 1))
i_ei_surr = np.zeros((surrogate_runs, 1))
e_i_surr = np.zeros((surrogate_runs, 1))
ei_surr = np.zeros((surrogate_runs, 1))
i_surr = np.zeros((surrogate_runs, 1))

for i in range(surrogate_runs):
    df_sample = df.sample(n=subsample_size) # subsample the data
    
    # run the surrogate models and store adj R^2
    
    # e, i, ei model
    Y_surr = df_sample['slope']
    X_surr = df_sample[['E', 'I', 'EI']]
    X_surr = add_constant(X_surr)
    ols_fit = linear_model.OLS(Y_surr, X_surr).fit()
    e_i_ei_surr[i] = ols_fit.rsquared_adj
    
    # i, ei model
    X_surr = df_sample[['I', 'EI']]
    X_surr = add_constant(X_surr)
    ols_fit = linear_model.OLS(Y_surr, X_surr).fit()
    i_ei_surr[i] = ols_fit.rsquared_adj
    
    # e, i model
    Y_surr = df_sample['slope']
    X_surr = df_sample[['E', 'I']]
    X_surr = add_constant(X_surr)
    ols_fit = linear_model.OLS(Y_surr, X_surr).fit()
    e_i_surr[i] = ols_fit.rsquared_adj

    # ei model
    X_surr = df_sample[['EI']]
    X_surr = add_constant(X_surr)
    ols_fit = linear_model.OLS(Y_surr, X_surr).fit()
    ei_surr[i] = ols_fit.rsquared_adj
    
    # i model
    X_surr = df_sample[['I']]
    X_surr = add_constant(X_surr)
    ols_fit = linear_model.OLS(Y_surr, X_surr).fit()
    i_surr[i] = ols_fit.rsquared_adj

In [334]:
# I v E_I
t_ei = sp.stats.ttest_rel(i_surr, e_i_surr)
np.shape(np.where((i_surr-e_i_surr)>0))[1], np.mean(i_surr), np.mean(e_i_surr), t_ei

(295,
 0.22195940249823343,
 0.23245647610541853,
 Ttest_relResult(statistic=array([-3.6005173]), pvalue=array([ 0.00034933])))

In [335]:
# I v I_EI
t_ei = sp.stats.ttest_rel(i_surr, i_ei_surr)
np.shape(np.where((i_surr-i_ei_surr)>0))[1], np.mean(i_surr), np.mean(i_ei_surr), t_ei

(298,
 0.22195940249823343,
 0.22833702617916757,
 Ttest_relResult(statistic=array([-2.81600109]), pvalue=array([ 0.00505503])))

In [341]:
# I_EI v E_I
t_ei = sp.stats.ttest_rel(i_ei_surr, e_i_surr)
np.shape(np.where((i_ei_surr-e_i_surr)>0))[1], np.mean(i_ei_surr), np.mean(e_i_surr), t_ei

(242,
 0.23598507034564312,
 0.23852512111522389,
 Ttest_relResult(statistic=array([-1.52694737]), pvalue=array([ 0.12740768])))

In [337]:
# I_EI v E_I_EI
t_ei = sp.stats.ttest_rel(i_ei_surr, e_i_ei_surr)
np.shape(np.where((i_ei_surr-e_i_ei_surr)>0))[1], np.mean(i_ei_surr), np.mean(e_i_ei_surr), t_ei

(356,
 0.22833702617916757,
 0.2210526850573063,
 Ttest_relResult(statistic=array([ 4.20426728]), pvalue=array([  3.10463839e-05])))

In [338]:
# E_I v E_I_EI
t_ei = sp.stats.ttest_rel(e_i_surr, e_i_ei_surr)
np.shape(np.where((e_i_surr-e_i_ei_surr)>0))[1], np.mean(e_i_surr), np.mean(e_i_ei_surr), t_ei

(378,
 0.23245647610541853,
 0.2210526850573063,
 Ttest_relResult(statistic=array([ 6.07189143]), pvalue=array([  2.50628967e-09])))